Install Beneath:

In [ ]:
!pip3 install --upgrade beneath

Create a CLI secret on [beneath.dev](https://beneath.dev) and use it authenticate Beneath:

In [ ]:
!beneath auth SECRET

Create a project

In [ ]:
!beneath project create USERNAME/NEW_PROJECT_NAME

Create a stream of movies

In [ ]:
import beneath
client = beneath.Client()

stream = await client.create_stream("USERNAME/PROJECT_NAME/NEW_STREAM_NAME", schema="""
    " A stream of movies "
    type Movie @schema {
        title: String! @key
        released_on: Timestamp! @key 
        director: String
        budget_usd: Int
        rating: Float
    }
""", update_if_exists=True)

Write a single record to the stream (head over to the web console to see records flow!)

In [ ]:
from datetime import datetime

async with stream.primary_instance.writer() as w:
    await w.write({
        "title":       "Star Wars: Episode IV",
        "released_on": datetime(year=1977, month=5, day=25),
        "director":    "George Lucas",
        "budget_usd":  11000000,
        "rating":      8.6,
    })

Write some more movies to the stream 

In [ ]:
import aiohttp
import asyncio

import random

# load a dataset of movies
url = "https://raw.githubusercontent.com/vega/vega/master/docs/data/movies.json"
async with aiohttp.ClientSession() as session:
    async with session.get(url) as res:
        movies = await res.json(content_type=None)

# write a 100 random movies one-by-one
n = 100
async with stream.primary_instance.writer() as w:
    for i in range(n):
        # get a random movie
        movie = random.choice(movies) 

        # transform and write the movie
        await w.write({
            "title":       str(movie["Title"]),
            "released_on": datetime.strptime(movie["Release Date"], "%b %d %Y"),
            "director":    movie["Director"],
            "budget_usd":  movie["Production Budget"],
            "rating":      movie["IMDB Rating"],
        })

        # wait about a second
        await asyncio.sleep(1.1)